# Introduction
This document will describe the steps that were taken to prepare the OHC (the AFIP Foundation) website data for analysis.

The data set contains data coming from an online field experiment that ran between May 23 until June 23 (2023). 12 different advertisements were distributed on Facebook and Instagram and sent people to a website on which different levels of web behavior and engagement was measured. After importing the individual level data through BigQuery, the data was cleaned and prepared with the code below.


In [ ]:
import pandas as pd
!pip install pyreadstat
import pyreadstat

df = pd.read_csv('YourFile')

The 'user_pseudo_id' variable needs to be transformed to an integer data type. And to better interpret the average engagement time on the web page, we also convert 'engagement_time_msec' into a new variable that represents seconds instead of milliseconds.

In [ ]:
#Converting 'user_pseudo_id' into an integer
df['user_pseudo_id'] = df['user_pseudo_id'].astype(int)
df.set_index('user_pseudo_id', inplace=True)

#Converting 'engagement_time_msec' to seconds
df['engagement_time_sec'] = df['engagement_time_msec'] / 1000


# Grouping data
We are interested in data related to the first visit of a website user. To make sure that the data set only includes users that came in through our campaign experiment, the page url should also only contain the campaign parameters. The following filter conditions are included:

*   First_visit == 1(True)
*   The page_url contains the number of the concepts that range from 1 to 12 (/concept-').


In [ ]:
grouped_data = df[(df['first_visit'] == 1) & (df['page_url'].str.contains('/concept-''))].groupby('user_pseudo_id').first()


The events that occured on the website need to be summed up per unique user before creating the dummies of whether an event occured (1= YES, 0= NO). These are the main events that could have occured per user:


*   email_nederlands: someone signed up to receive emails
*   community_aanmelding: someone signed up to become part of the community
*   contact_nederlands: someone requested contact
*   account_registratie: someone created a website / forum account
*   session_engaged: a session that lasted longer than 10 seconds, has a conversion event, or has at least 2 page- or screenviews.







In [ ]:
sum_columns = ['email_nederlands', 'community_aanmelding', 'contact_nederlands', 'account_registratie', 'session_engaged']
sum_data = df.groupby('user_pseudo_id')[sum_columns].sum()
sum_data.columns = [f'sum_{col}' for col in sum_data.columns]
final_data = grouped_data.merge(sum_data, left_index=True, right_index=True)

# Creating dummy variables

In this data set, multiple dummy variables are created. This section will explain the definition of the dummy variables, followed by the code.

**Main variable dummies**

Dummies are assigned based on the page URL that was visited by a unique user during the first visit. The experiment contains 12 different communication concepts. The concepts (i.e., concepts 1 to 12) contain differtent emotions, topics, appeal and linguistic style. Each concept has its own URL path on which the concept variables are based :

*   Concept 1: contains 'concept-1/' in URL
*   Concept 2: contains 'concept-2/' in URL
*   Concept 3: contains 'concept-3/' in URL
*   Concept 4: contains 'concept-4/' in URL
*   Concept 5: contains 'concept-5/' in URL
*   Concept 6: contains 'concept-6/' in URL
*   Concept 7: contains 'concept-7/' in URL
*   Concept 8: contains 'concept-8/' in URL
*   Concept 9: contains 'concept-9/' in URL
*   Concept 10: contains 'concept-10/' in URL
*   Concept 11: contains 'concept-11/' in URL
*   Concept 12: contains 'concept-12/' in URL







In [ ]:
def generate_dummies(page_url):
    dummies = {
        'Emotion_Love':0,
        'Emotion_Fear': 0,
        'Topic_Sprotection':0,
        'Topic_Affiliation': 0,
        'Topic_Kincare': 0,
        'Appeal_Exp':0,
        'Appeal_Testi': 0,
        'Appeal_Infor':0,
        'Appeal_Pers': 0,
        'LStyle_Fperson':0,
        'LStyle_Tperson': 0,
    }


    if 'concept-1/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Kincare'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Fperson'] = 1
    elif 'concept-2/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Sprotection'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-3/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Affiliation'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-4/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Sprotection'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Fperson'] = 1
    elif 'concept-5/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Kincare'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Fperson'] = 1
    elif 'concept-6/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Affiliation'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-7/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Kincare'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-8/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Affiliation'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Fperson'] = 1
    elif 'concept-9/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Sprotection'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Testi'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Fperson'] = 1
    elif 'concept-10/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Kincare'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Infor'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-11/' in page_url:
        dummies['Emotion_Love'] = 1
        dummies['Topic_Affiliation'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Tperson'] = 1
    elif 'concept-12/' in page_url:
        dummies['Emotion_Fear'] = 1
        dummies['Topic_Sprotection'] = 1
        dummies['Appeal_Exp'] = 1
        dummies['Appeal_Pers'] = 1
        dummies['LStyle_Fperson'] = 1

    return dummies

**Concept label dummies**

This function creates a variable, named 'concepts', that will label the concept numbers based on the URL paths.

In [ ]:
def generate_concept(page_url):

    elif 'concept-1/' in page_url:
        return 1
    elif 'concept-2/' in page_url:
        return 2
    elif 'concept-3/' in page_url:
        return 3
    elif 'concept-4/' in page_url:
        return 4
    elif 'concept-5/' in page_url:
        return 5
    elif 'concept-6/' in page_url:
        return 6
    elif 'concept-7/' in page_url:
        return 7
    elif 'concept-8/' in page_url:
        return 8
    elif 'concept-9/' in page_url:
        return 9
    elif 'concept-10/' in page_url:
        return 10
    elif 'concept-11/' in page_url:
        return 11
    elif 'concept-12/' in page_url:
        return 12
    else:
        return 0


The next step is to apply the results from the functions to the dataframe.

In [ ]:
final_data['dummies'] = grouped_data['page_url'].apply(generate_dummies)
final_data['concept'] = grouped_data['page_url'].apply(generate_concept)

page_url_dummies = final_data.groupby('user_pseudo_id')['page_url'].first().apply(generate_dummies).apply(pd.Series)

final_data = final_data.join(page_url_dummies)


**Engagement time dummies**

The original engagement_time_msec variable takes the engagement time in milliseconds of the first visit. To also take other engagement time into consideration, extra variables are created to get more insights in the avarage and total time someone spent on the website. The following dummies are generated:


*   avg_engagement_time_msec: the average time a user spent during all of the website sessions that took place between May 23 to June 23 (2023).
*   total_engagement_time_msec:  the total time a user spent during all of the website sessions that took place between May 23 to June 23 (2023).
*   user_engaged_time: users that were not marked as engaged (i.e., session_engaged == 0) during the first visit, but still had some milliseconds of engagement registered.





In [ ]:
user_session_avg_engagement_time = df.groupby(['user_pseudo_id', 'session_id'])['engagement_time_sec'].mean()
user_total_avg_engagement_time = user_session_avg_engagement_time.reset_index().groupby('user_pseudo_id')['engagement_time_sec'].sum()
final_data['total_engagement_time_sec'] = user_total_avg_engagement_time

# Calculating the average engagement time for each unique session_id and user_pseudo_id
session_avg_engagement_time = df.groupby(['user_pseudo_id', 'session_id'])['engagement_time_sec'].mean()
user_avg_engagement_time = session_avg_engagement_time.reset_index().groupby('user_pseudo_id')['engagement_time_sec'].mean()

final_data['avg_engagement_time_sec'] = user_avg_engagement_time


def calculate_user_engaged_time(row):
    if row['session_engaged'] == 0 and row['engagement_time_sec'] > 0:
        return 1  # YES
    else:
        return 0  # NO

final_data['user_engaged_time'] = final_data.apply(calculate_user_engaged_time, axis=1)

**Time related dummy variables**

The time related dummy variables are created based on the date variable. A 1= YES or 0= NO value is attributed to the following time dimensios:


*Part of week (PW)*
*   Week: a day in the (work) week (reference: coded as 0)
*   Weekend: a day in the weekend (coded as 1)

*Part of day (PD)*
* Morning: time between 06:00 and 12:00  (reference: coded as 000)
* Afternoon: time between 12:00 and 18:00  (coded as 100)
* Evening: time between 18:00 and 12:00 (coded as 010)
* Night: time between 00:00 and 06:00   (coded as 001)

*Working hours (WH)*


*   Non working hours: time between 18:00 and 09:00 (reference: coded as 0)
*   Working hours: time between 09:00 and 18:00 (coded as 1)

*Date to day number*

The date is also transformed into the day number of the campaign length. The campaign ran from May 23 until June 23 (2023). Thus, May 23 is marked as day 1 and June 23 as day 32.










In [ ]:
# Creating the part of week dummies (week=0, weekend=1)
final_data['PW_week_dummy'] = (final_data['part_of_week'] == 'week').astype(int)
final_data['PW_weekend_dummy'] = (final_data['part_of_week'] == 'weekend').astype(int)

# Creating the part of day dummies (morning= 0, others =1)
final_data['PD_morning_dummy'] = (final_data['part_of_day'] == 'morning').astype(int)
final_data['PD_afternoon_dummy'] = (final_data['part_of_day'] == 'afternoon').astype(int)
final_data['PD_evening_dummy'] = (final_data['part_of_day'] == 'evening').astype(int)
final_data['PD_night_dummy'] = (final_data['part_of_day'] == 'night').astype(int)

# Creating working hours dummies (non working hours=0, working hours=1)
final_data['WH_nonworking_dummy'] = (final_data['working_hours'] == 'non-working hours').astype(int)
final_data['WH_workhours_dummy'] = (final_data['working_hours'] == 'working hours').astype(int)

final_data['date'] = pd.to_datetime(final_data['date'], format='%Y%m%d')

start_date = pd.Timestamp("2023-05-23")
end_date = pd.Timestamp("2023-06-23")


final_data['date_dummy'] = (final_data['date'] - start_date).dt.days + 1

**Event dummies**

The following dummies represent the events that occured during the website visits of users. Again, 1= YES, 0= NO.

* Email_dummy: someone signed up to receive emails
*   ComSub_dummy: someone signed up to become part of the community
*   Contact_dummy: someone requested contact
*   Account_dummy: someone created a website / forum account
*   Engaged_All_Sessions: someone engaged* at least in one website session
*   Engaged_Not_First_Visit: someone engaged* in at least one website session, but not during the first visit.

*Engaged session: a session that lasted longer than 10 seconds, has a conversion event, or has at least 2 page- or screenviews.





In [ ]:
final_data['Email_dummy'] = (final_data['sum_email_nederlands'] > 0).astype(int)
final_data['ComSub_dummy'] = (final_data['sum_community_aanmelding'] > 0).astype(int)
final_data['Contact_dummy'] = (final_data['sum_contact_nederlands'] > 0).astype(int)
final_data['Account_dummy'] = (final_data['sum_account_registratie'] > 0).astype(int)
final_data['Engaged_All_Sessions'] = (final_data['sum_session_engaged'] > 0).astype(int)
final_data['Engaged_Not_First_Visit'] = ((final_data['first_visit'] == 0) & (final_data['sum_session_engaged'] > 0)).astype(int)  # Use 'sum_' prefix


**Device Category dummies**

The following dummies allocate the device, which the website user used to visit the website, to one of the following categories:

*   Mobile (reference: coded as 00)
*   Desktop (coded as 10)
*   Tablet (coded as 01)






In [ ]:
final_data['DC_desktop_dummy'] = (final_data['device_category'] == 'desktop').astype(int)
final_data['DC_tablet_dummy'] = (final_data['device_category'] == 'tablet').astype(int)
final_data['DC_mobile_dummy']= (final_data['device_category'] == 'mobile').astype(int)

#Column selection
The following columns are selected to be included within the data set.

In [ ]:
selected_columns = [
    'date', 'date_dummy', 'session_id', 'first_visit_date', 'event_timestamp','part_of_week', 'PW_week_dummy','PW_weekend_dummy', 'part_of_day', 'PD_morning_dummy','PD_afternoon_dummy', 'PD_evening_dummy','PD_night_dummy','working_hours','WH_nonworking_dummy', 'WH_workhours_dummy','page_url', 'device_category','DC_mobile_dummy','DC_desktop_dummy','DC_tablet_dummy',
    'city', 'campaign', 'concept', 'Emotion_Love','Emotion_Fear', 'Topic_Sprotection','Topic_Affiliation',
    'Topic_Kincare','Appeal_Exp', 'Appeal_Testi','Appeal_Infor', 'Appeal_Pers','LStyle_Fperson', 'LStyle_Tperson', 'first_visit',
    'Email_dummy', 'ComSub_dummy', 'Contact_dummy', 'Account_dummy', 'session_engaged','Engaged_All_Sessions', 'Engaged_Not_First_Visit', 'engagement_time_msec','engagement_time_sec', 'avg_engagement_time_sec', 'total_engagement_time_sec','user_engaged_time'
]

In [ ]:
final_data = final_data[selected_columns].reset_index()

# Removing missing values
Some variables left some blanks. Blanks for numeric variables are replaced with '0'. Missing values for object types, such as City, are replaced with 'unknown'.

In [ ]:
final_data['engagement_time_sec'] = pd.to_numeric(final_data['engagement_time_sec'], errors='coerce').fillna(0).astype(int)
final_data['avg_engagement_time_sec'] = pd.to_numeric(final_data['avg_engagement_time_sec'], errors='coerce').fillna(0).astype(int)
final_data['total_engagement_time_sec'] = pd.to_numeric(final_data['total_engagement_time_sec'], errors='coerce').fillna(0).astype(int)

final_data['city'] = final_data['city'].astype(str).apply(lambda x: x.strip())
final_data['city'].replace('', 'Unknown', inplace=True)

# Saving the file to .csv



In [ ]:
final_data.to_csv('YourFile', index=False)

# Saving the file to SPSS format (if needed)
pyreadstat.write_sav(final_data, 'YourFile')